In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

## Project 1 - NLP and Text Classification

For this project you will need to classify some angry comments into their respective category of angry. The process that you'll need to follow is (roughly):
<ol>
<li> Use NLP techniques to process the training data. 
<li> Train model(s) to predict which class(es) each comment is in.
    <ul>
    <li> A comment can belong to any number of classes, including none. 
    </ul>
<li> Generate predictions for each of the comments in the test data. 
<li> Write your test data predicitions to a CSV file, which will be scored. 
</ol>

You can use any models and NLP libraries you'd like. Think aobut the problem, look back to see if there's anything that might help, give it a try, and see if that helps. We've regularly said we have a "toolkit" of things that we can use, we generally don't know which ones we'll need, but here you have a pretty simple goal - if it makes it more accurate, it helps. There's not one specific solution here, there are lots of things that you could do. 

## Training Data

Use the training data to train your prediction model(s). Each of the classification output columns (toxic to the end) is a human label for the comment_text, assessing if it falls into that category of "rude". A comment may fall into any number of categories, or none at all. Membership in one output category is <b>independent</b> of membership in any of the other classes (think about this when you plan on how to make these predictions - it may also make it easier to split work amongst a team...). 

In [2]:
train_df = pd.read_csv("train.csv.zip")
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def preprocess_text(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    words = [word for word in words if not word in set(stopwords.words("english"))]
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    cleaned_text = " ".join(stemmed_words)
    return cleaned_text

In [19]:
train_df['cleaned_text'] = train_df['comment_text'].apply(preprocess_text)

KeyboardInterrupt: 

Reshapes the target labels from the dataframe into 2D numpy arrays with one column and multiple rows, and stores them in a list to be used as the target variables for modeling.

In [4]:
y_toxic = np.array(train_df["toxic"]).reshape(-1,1)
y_stoxic= np.array(train_df["severe_toxic"]).reshape(-1,1)
y_obscene = np.array(train_df["obscene"]).reshape(-1,1)
y_threat = np.array(train_df["threat"]).reshape(-1,1)
y_insult = np.array(train_df["insult"]).reshape(-1,1)
y_idhate = np.array(train_df["identity_hate"]).reshape(-1,1)
list_targets = [y_toxic, y_stoxic, y_obscene, y_threat, y_insult, y_idhate]

## toxic

In [5]:
np.unique(y_toxic, return_counts=True)

(array([0, 1], dtype=int64), array([144277,  15294], dtype=int64))

In [16]:
#NB Model
#vector_nb_1 = TfidfVectorizer()
#model_nb_1 = MultinomialNB()
model_nb_1 = RandomForestClassifier()
stemmer_1 = PorterStemmer()
#stemmer_1 = SnowballStemmer()
#lemmatizer = WordNetLemmatizer()
vector_nb_1 = TfidfVectorizer()
X_toxic = train_df["comment_text"]
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_toxic, y_toxic)

# Use original data
pipe_nb_1 = Pipeline([("vect", vector_nb_1),("model", model_nb_1)])
# Use oversample
#pipe_nb_1 = make_pipeline(vector_nb_1, oversample, model_nb_1)

pipe_nb_1.fit(X_train_1, y_train_1.ravel())
preds_1 = pipe_nb_1.predict(X_test_1)
#pipe_nb.score(X_test,y_test)
print(classification_report(y_test_1, preds_1))
confusion_matrix(y_test_1, preds_1)


pipe_nb_1.fit(X_toxic, y_toxic.ravel())

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
stemmer_1 = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer_1.stem(word) for word in text.split()])

vector_nb_1 = TfidfVectorizer(preprocessor=stem_words)

model_nb_1 = RandomForestClassifier()

X_toxic = train_df["comment_text"]
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_toxic, y_toxic)

pipe_nb_1 = Pipeline([("vect", vector_nb_1),("model", model_nb_1)])

pipe_nb_1.fit(X_train_1, y_train_1.ravel())
preds_1 = pipe_nb_1.predict(X_test_1)
print(classification_report(y_test_1, preds_1))
confusion_matrix(y_test_1, preds_1)

pipe_nb_1.fit(X_toxic, y_toxic.ravel())

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     36023
           1       0.89      0.40      0.55      3870

    accuracy                           0.94     39893
   macro avg       0.92      0.70      0.76     39893
weighted avg       0.93      0.94      0.93     39893

In [14]:
y_toxic2 = np.array(train_df["toxic"]).reshape(-1,1)
X_toxic2 = train_df["clean_text"]
X_train_toxic2, X_test_toxic2, y_train_toxic2, y_test_toxic2 = train_test_split(X_toxic2, y_toxic2)

pipe_nb_1.fit(X_train_toxic2, y_train_toxic2.ravel())
preds_toxic2 = pipe_nb_1.predict(X_test_toxic2)
print(classification_report(y_test_toxic2, preds_toxic2))
confusion_matrix(y_test_toxic2, preds_1)


pipe_nb_1.fit(X_toxic, y_toxic.ravel())

AttributeError: 'list' object has no attribute 'lower'

## Test Data

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

## Output Details, Submission Info, and Example Submission

For this project, please output your predictions in a CSV file. The structure of the CSV file should match the structure of the example below. 

The output should contain one row for each row of test data, complete with the columns for ID and each classification.

Into Moodle please submit:
<ul>
<li> Your notebook file(s). I'm not going to run them, just look. 
<li> Your sample submission CSV. This will be evaluated for accuracy against the real labels; only a subset of the predictions will be scored. 
</ul>

It is REALLY, REALLY, REALLY important the the structure of your output matches the specifications. The accuracies will be calculated by a script, and it is expecting a specific format. 

### Sample Evaluator

The file prediction_evaluator.ipynb contains an example scoring function, scoreChecker. This function takes a sumbission and an answer key, loops through, and evaluates the accuracy. You can use this to verify the format of your submission. I'm going to use the same function to evaluate the accuracy of your submission, against the answer key (unless I made some mistake in this counting function).

In [ ]:
#Construct dummy data for a sample output. 
#You won't do this part first, you have real data - I'm faking it. 
#Your data should have the same structure, so the CSV output is the same
dummy_ids = ["dfasdf234", "asdfgw43r52", "asdgtawe4", "wqtr215432"]
dummy_toxic = [0,0,0,0]
dummy_severe = [0,0,0,0]
dummy_obscene = [0,1,1,0]
dummy_threat = [0,1,0,1]
dummy_insult = [0,0,1,0]
dummy_ident = [0,1,1,0]
columns = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_out = pd.DataFrame( list(zip(dummy_ids, dummy_toxic, dummy_severe, dummy_obscene, dummy_threat, dummy_insult, dummy_ident)),
                    columns=columns)
sample_out.head()

In [ ]:
#Write DF to CSV. Please keep the "out.csv" filename. Moodle will auto-preface it with an identifier when I download it. 
#This command should work with your dataframe of predictions. 
sample_out.to_csv('out.csv', index=False)  

## Grading

The grading for this is split between accuracy and well written code:
<ul>
<li> 75% - Accuracy. The most accurate will get 100% on this, the others will be scaled down from there. 
<li> 25% - Code quality. Can the code be followed and made sense of - i.e. comments, sections, titles. 
</ul>